## Importing Libs

In [1]:
# Standard libs
import numpy as np
import pandas as pd

/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Loading Data

In [ ]:
# Load data

root_folder = '/Users/dellacorte/py-projects/data-science/'

df_abt = pd.read_csv(f'{raiz_folder}/propensao_revenda_abt.csv')

# get the training base
df_train = df_abt.query('data_ref_safra < "2018-03-01"')

# get the evaluation base (out of time)
df_oot = df_abt.query('data_ref_safra == "2018-03-01"')